In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from imblearn.pipeline import Pipeline as imPipeline
from imblearn.under_sampling import RandomUnderSampler, TomekLinks
from imblearn.over_sampling import RandomOverSampler
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import LabelEncoder

from spacy.lang.en.stop_words import STOP_WORDS


Bad val 'GTKAgg' on line #32
	"backend      : GTKAgg
"
	in file "/home/alex/.local/lib/python3.7/site-packages/matplotlib/mpl-data/matplotlibrc"
	Key backend: Unrecognized backend string 'gtkagg': valid strings are ['GTK3Agg', 'GTK3Cairo', 'MacOSX', 'nbAgg', 'Qt4Agg', 'Qt4Cairo', 'Qt5Agg', 'Qt5Cairo', 'TkAgg', 'TkCairo', 'WebAgg', 'WX', 'WXAgg', 'WXCairo', 'agg', 'cairo', 'pdf', 'pgf', 'ps', 'svg', 'template']
/home/alex/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/alex/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", 

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
# For Google colab only

df = pd.read_csv("../new_clean_2million.csv")
df = df[df['reviewText'].notna()]


/home/alex/anaconda3/envs/deepLearning/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Split data

In [3]:
X = df.reviewText.values
y = df.overall.values

In [4]:
num = len(df)
X, y  = X[:num], y[:num]
le = LabelEncoder()
y = le.fit_transform(y)

In [5]:
#X_train, X_test, y_train, y_test = train_test_split(X_rus, y_rus, test_size=0.33, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [6]:

def train_predict(pipline):
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    score = classification_report(y_test, y_pred)
    print(score)


Vectorizers


In [7]:
count_vectoriser = Pipeline([
                ('countVectoriser', CountVectorizer())
            ])

tfidf_vectoriser = Pipeline([
                ('tfidfVectoriser', TfidfVectorizer(stop_words=STOP_WORDS))
            ])


In [8]:
class TextStats(TransformerMixin, BaseEstimator):
    """Extract features from each document for DictVectorizer"""

    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        return [{'length': len(text),
                 'num_sentences': text.count('.')}
                for text in posts]



In [9]:
def make_balance_pipe(steps):
    
    
    x = list(range(len(steps)))
    pipeline= [(str(x),y) for x,y in zip(x,steps)]

    return imPipeline(pipeline)


In [10]:
text_features = Pipeline([('length',TextStats()),
         ('vect', DictVectorizer())
         ])



Naive Bayes


In [11]:


naive_bayes = Pipeline([
    ('classifier', MultinomialNB()),
])

pipeline = Pipeline([
    ('vectoriser', tfidf_vectoriser),
    ('classifier', naive_bayes)
])

pipeline = make_balance_pipe([TfidfVectorizer(),
                              RandomUnderSampler(),
                              MultinomialNB()])


pipeline = Pipeline([
    ('features',FeatureUnion([
                ('text_stats', text_features),
                ('ngram_tfidf',tfidf_vectoriser)
            ])),
    ('classifier',MultinomialNB())
])

In [12]:
train_predict(pipeline)


/home/alex/anaconda3/envs/deepLearning/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


              precision    recall  f1-score   support

           0       0.58      0.56      0.57    162661
           1       0.41      0.47      0.43    163253
           2       0.46      0.37      0.41    162950
           3       0.46      0.40      0.43    161697
           4       0.56      0.70      0.62    161776

    accuracy                           0.50    812337
   macro avg       0.50      0.50      0.49    812337
weighted avg       0.50      0.50      0.49    812337

